In [2]:
import cv2
import pytesseract
import re

# Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Compile regular expressions for PKD, Expiry Date, and Price
pkd_regex = re.compile(r'(PKD|Pack(ed)? Date|Packing Date|Manufacture Date):?\s*([\d/.-]+)', re.IGNORECASE)
expiry_regex = re.compile(r'(EXP|Expiry Date|Expiration):?\s*([\d/.-]+)', re.IGNORECASE)
price_regex = re.compile(r'(Price|Cost|Mrp|M.R.P.|Rs.|₹|\$):?\s*\$?([\d,]+(\.\d{2})?)', re.IGNORECASE)

# Dictionary to store unique detected products (keyed by price)
unique_products = {}
total_price = 0.0  # Variable to store the total price

while True:
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Convert the frame to grayscale (optional, but can improve OCR accuracy)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform OCR using pytesseract
    result = pytesseract.image_to_string(gray)

    # Variables to store detected values
    pkd_text, expiry_text, price_text = None, None, None

    # Extract PKD, Expiry Date, and Price using regular expressions
    pkd_match = pkd_regex.search(result)
    expiry_match = expiry_regex.search(result)
    price_match = price_regex.search(result)

    # Prepare the y-offset for displaying on the frame
    y_offset = 50

    # If PKD is detected
    if pkd_match:
        pkd_text = f"PKD: {pkd_match.group(3)}"
        cv2.putText(frame, pkd_text, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        y_offset += 30  # Move the next line down

    # If Expiry Date is detected
    if expiry_match:
        expiry_text = f"Expiry: {expiry_match.group(2)}"
        cv2.putText(frame, expiry_text, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        y_offset += 30

    # If Price is detected, show it in the camera feed
    if price_match:
        price_text = f"Price: ₹{price_match.group(2)}"
        cv2.putText(frame, price_text, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        y_offset += 30

        # Convert price to float and update total
        try:
            price_value = float(price_match.group(2).replace(",", ""))

            # Only add the product if it's not already in the unique_products dictionary
            product_key = (price_value, expiry_text)  # Tuple with price and expiry to identify a unique product

            if product_key not in unique_products:
                unique_products[product_key] = {
                    'PKD': pkd_text if pkd_text else "N/A",
                    'Expiry': expiry_text if expiry_text else "N/A",
                    'Price': price_value
                }
                total_price += price_value

        except ValueError:
            print("Error: Could not convert price to a float.")

    # Show the frame with OCR results
    cv2.imshow('OCR on Webcam Feed', frame)

    # Press 'q' to quit the loop and close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

# Print detected unique products in an ordered list
if unique_products:
    print("\nDetected Products:")
    for i, (key, product) in enumerate(unique_products.items(), 1):
        # Print PKD, Expiry, and Price for each product
        print(f"{i}. {product['PKD']}, {product['Expiry']}, Price: ₹{product['Price']:.2f}")

    # Print the total price
    print(f"\nTotal Price: ₹{total_price:.2f}")
else:
    print("No products detected.")


Detected Products:
1. N/A, N/A, Price: ₹8.00
2. N/A, N/A, Price: ₹99.00
3. N/A, N/A, Price: ₹999.00
4. N/A, N/A, Price: ₹140.00
5. N/A, N/A, Price: ₹1699.00

Total Price: ₹2945.00
